# Задание 1 ещё в процессе

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [103]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [3]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [4]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [106]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату.

In [107]:
def foldl2(f, x0, lst):
    return foldr(lambda  x,y: lambda z: y(f(z,x)) , lambda x: x, lst)(x0)

foldl2(f, 1, [1,2,3])

0.16666666666666666

In [109]:
def foldr2(f, x0, lst):
    return foldl(lambda  x,y: lambda z: y(f(z,x)) , lambda x: x, lst)(x0)
foldr2(f, 1, [1, 2, 3])

TypeError: unsupported operand type(s) for /: 'int' and 'function'

# Задание 2 готово

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [51]:
def check_inv(a, b):
    if len(a)<len(b):
        return False
    #вспомогательные данные: считаем буквы в слове b
    bDict = {}
    for letterB in b:                                                        #o(b)
        try:
            bDict[letterB] = bDict[letterB] + 1                                 #o(1)
        except:
            bDict[letterB] = 1                                                  #o(1)
            
                                                                             #итого o(b) 
                
    #пытаемся занулить словарь подряд стоящими буквами в слове а. успешность процедуры есть результат метода
    
    tempBDict = bDict.copy()                                                  #o(b)
    for letterA in a:                                                         #o(a)
        if letterA in tempBDict:                                                 #o(b)
            if tempBDict[letterA] > 0:                                              #o(1)
                tempBDict[letterA] = tempBDict[letterA] - 1                         #o(1)
                if all(value == 0 for value in tempBDict.values()):                    #o(b)
                    return True
        else:
            tempBDict = bDict.copy()                                           #o(b)
                                                                               #итого o(a*b^2)
    return False        

In [100]:
a = 'коллаборация'
b = 'балл'
print(check_inv(a, b))

True


11

# Задание 3 готово

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [95]:
def searchLeaves(tree, leaves):
    if (tree.left != None):
        searchLeaves(tree.left, leaves)
    if (tree.right != None):
        searchLeaves(tree.right, leaves)
    if (tree.left == None and tree.right == None):
        leaves.append(tree.value)
            
def takeNodes(tree, values):        
    values.append(str(tree.value))
    if (tree.left != None):
        values.append('l')
        takeNodes(tree.left, values)            
    if (tree.right != None):
        values.append('r')
        takeNodes(tree.right, values)
    values.append('up')
        
class Tree:        
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        leaves = []
        searchLeaves(self, leaves)
        return iter(leaves)
    
    def __str__(self):
        values = []
        takeNodes(self, values)
        rep = ' '.join(values)
        i = 0
        res = []
        for a in rep:
            if a == 'l':
                i = i+1
            elif a == 'r':
                i = i+1
            elif a == 'u':
                i = i-1
            elif a == 'p' or a == ' ':
                continue
            else:
                try:
                    res[i] = res[i] + ' ' + a
                except:                    
                    res.insert(i, str(a))
        ress = ''
        for line in res:
            ress = ress + '\n' + line 
        return ress
    
    def __repr__(self):
        values = []
        takeNodes(self, values)
        return ' '.join(values)
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

#tree = Tree(1, 
#            Tree(2, 
#                 Tree(3), 
#                 Tree(4, 
#                      Tree(5, 
#                           Tree(7)), 
#                      Tree(6))), 
#                 Tree(8, None, 
#                      Tree(9, 
#                           Tree(10, 
#                                Tree(11, None, 
#                                     Tree(12))))))

list(tree) == [3, 4, 2]

True

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

In [176]:
def isInt(syms):
    try:
        int(syms)
        return True
    except:
        return False

def toReversePolishNotation(expr):
    expr = expr.replace('(', '( ').replace(')', ' )')
    queue = expr.split(' ')
    res = ''
    stack = []
    for sym in queue:
        if isInt(sym):
            res = res + sym + ' '
        elif sym == '+' or sym == '-':
            while stack and (stack[-1] == '*' or stack[-1] == '\\'):
                res = res + stack.pop() + ' '
            stack.append(sym)
        elif sym == '*' or sym == '\\':
            stack.append(sym)                
        elif sym == '(':
            stack.append(sym)
        elif sym == ')':
            while stack[-1] != '(':
                res = res + stack.pop() + ' '
            stack.pop()
    while stack:
        res = res + stack.pop() + ' '
    return res[:-1]
                
        
def calc(expr):
    rpn = toReversePolishNotation(expr)
    syms = rpn.split(' ')
    i = 0
    while len(syms)>1:
        if isInt(syms[i]):
            i = i + 1
        else:
            if syms[i] == '+':
                syms[i-2] = int(syms[i-2]) + int(syms[i-1])
            if syms[i] == '-':
                syms[i-2] = int(syms[i-2]) - int(syms[i-1])
            if syms[i] == '*':
                syms[i-2] = int(syms[i-2]) * int(syms[i-1])
            if syms[i] == '\\':
                syms[i-2] = int(syms[i-2]) / int(syms[i-1])            
            del syms[i-1]
            del syms[i-1]
            i = i - 1
    return syms[-1]

(calc('2 * (15 - 3 * 4) - 2') == 4, 
calc('(1 + 2) * 4 + 3') == 15,
calc('(4 + 4) \ 4') == 2,
calc('4 + 4 \ 4') == 5)

(True, True, True, True)

In [156]:
lst = [2, 3, 4, 5, 6, 7]
del lst[2]
del lst[2]
lst[2] = '*'
lst

[2, 3, '*', 7]